In [ ]:
import tkinter as tk
import requests
from urllib.parse import urlparse
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from flask import Flask, render_template, request

app = Flask(__name__)

def evaluate_complexity(repository_name):
    # Initialize tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    
    # Prompt engineering
    prompt = f"Evaluate the technical complexity of the repository: {repository_name}. Analyze the code and provide insights on its complexity."
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, truncation=True, max_length=100, return_tensors="pt")
    output = model.generate(input_ids=input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id)
    
    complexity_score = calculate_complexity_score(output)
    return complexity_score
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
def calculate_complexity_score(output):
    # Process the GPT output to derive the complexity score
    processed_output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Count the number of words in the processed output
    word_count = len(processed_output.split())
    
    # Define a mapping between word count and complexity score
    max_complexity_score = 10000
    
    # Determine the complexity score based on the output length
    complexity_score = min(word_count, max_complexity_score)
    
    return complexity_score
    
    


@app.route('/', methods=['GET', 'POST'])
def fetch_github_repositories():
    if request.method == 'POST':
        user_url = request.form['user_url']
        
        # Extracting the GitHub username from the user URL
        parsed_url = urlparse(user_url)
        username = parsed_url.path.strip("/")
        
        if not username:
            return render_template('index.html', error="Invalid GitHub user URL.")
        
        # API endpoint to fetch user repositories
        api_url = f"https://api.github.com/users/{username}/repos"
        
        try:
            response = requests.get(api_url)
            response.raise_for_status()
            repositories = response.json()
            
            # Process repositories
            if repositories:
                repository_names = [repo["name"] for repo in repositories]
                longest_repo_name = max(repository_names, key=len)
                most_complex_repo = evaluate_complexity(longest_repo_name)
                
                return render_template('index.html', most_complex_repo=longest_repo_name, complexity_score=most_complex_repo)
            else:
                return render_template('index.html', error="No repositories found.")
        except requests.exceptions.HTTPError as err:
            return render_template('index.html', error=f"An HTTP error occurred: {err}")
        except requests.exceptions.RequestException as err:
            return render_template('index.html', error=f"An error occurred: {err}")
    
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jun/2023 22:34:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 22:34:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 22:35:18] "POST / HTTP/1.1" 200 -
